In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential, Model
from keras import optimizers
from keras.applications import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

plt.switch_backend('agg')

Using TensorFlow backend.


In [2]:
EPOCHS = 10
N_LABELS = 10
BATCH_SIZE = 64
RESOLUTION = 299
EXP_NAME = 'e001'
PATH_TRAIN = '/home/dieinison/script/dataset/faces_split/train/'
PATH_VAL = '/home/dieinison/script/dataset/faces_split/val/'

In [3]:
def overfitting_plot(history, name):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend(loc='best')
    
    plt.savefig('./plots/acc_'+name+'.png')
    
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend(loc='best')
    
    plt.savefig('./plots/loss_'+name+'.png')

In [ ]:
def train(model, path_train, path_val, name=''):
    
    train_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        path_train,
        target_size=(RESOLUTION, RESOLUTION),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
    N_TRAIN = len(train_generator.filenames)
    
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    val_generator = val_datagen.flow_from_directory(
        path_val,
        target_size=(RESOLUTION, RESOLUTION),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
    N_VAL = len(val_generator.filenames)
    
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=(N_TRAIN // BATCH_SIZE),
        epochs=EPOCHS,
        validation_data=val_generator ,
        validation_steps=(N_VAL // BATCH_SIZE)
    )
    
    model.save(name+'.h5')
    
    overfitting_plot(history, name)

In [ ]:
def build_inceptionV3(learning_rate=0.0001):
    
    model = InceptionV3(include_top=False,
                        input_shape=(RESOLUTION,RESOLUTION,3),
                        weights='imagenet')
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(1024, activation='relu')(x)
    
    predictions = Dense(N_LABELS, activation='softmax')(x)

    model = Model(inputs=base_model.input,               
                  outputs=predictions)
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=learning_rate),
                  metrics=['acc'])
    
    return model

In [ ]:
inception_v3_model = build_inceptionV3(learning_rate=0.0001)

In [ ]:
try:
    train(inception_v3_model, PATH_TRAIN, PATH_VAL, name=EXP_NAME)
except Exception as e:
    print(str(e))